In [1]:
!pip install numpy
!pip install pandas
!pip install mealpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 90.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import time
from mealpy.utils.space import IntegerVar
from mealpy.swarm_based import PSO,MPA # Example algorithm (you can replace this with others like GA, PSO)
from mealpy.bio_based import WHO # Example algorithm (you can replace this with others like GA, PSO)
from mealpy.physics_based import NRO
# Function to load datasets from CSV
def load_datasets(size="Small"):
    # Load the previously saved datasets for each warehouse size
    items_df = pd.read_csv(f'{size}_items_dataset.csv')
    slots_df = pd.read_csv(f'{size}_slots_dataset.csv')
    print(f"Datasets loaded for {size} warehouse.")
    return items_df, slots_df

# Fitness function (same as before)
def fitness_function(solution, items_df, slots_df):
    total_distance = 0
    penalty = 0
    used_slots = set()

    for i, slot_index in enumerate(solution):
        item = items_df.iloc[i]
        if slot_index >= len(slots_df):
            penalty += 10
            continue

        slot = slots_df.iloc[int(slot_index)]

        # Add penalty if slot already used
        if slot["SlotID"] in used_slots:
            penalty += 5
        else:
            used_slots.add(slot["SlotID"])

        # Check size compatibility
        if item["Size"] == "Small" and "Small" not in slot["SlotType"] and "Standard" not in slot["SlotType"]:
            penalty += 3
        elif item["Size"] == "Medium" and "Medium" not in slot["SlotType"] and "Standard" not in slot["SlotType"]:
            penalty += 3
        elif item["Size"] == "Large" and "Large" not in slot["SlotType"] and "Standard" not in slot["SlotType"]:
            penalty += 3

        # Add distance based on proximity
        if item["DemandFrequency"] == "High":
            total_distance += 1 if slot["Proximity"] == "Close" else 5
        elif item["DemandFrequency"] == "Medium":
            total_distance += 3 if slot["Proximity"] == "Medium" else 5
        elif item["DemandFrequency"] == "Low":
            total_distance += 5 if slot["Proximity"] == "Far" else 10

    return total_distance + penalty

# Function to optimize allocation with different algorithms
def optimize_allocation(items_df, slots_df, num_items, num_slots, algorithm="NRO"):
    bounds = [IntegerVar(0, num_slots - 1) for _ in range(num_items)]
    problem = {
        "obj_func": lambda solution: fitness_function(solution, items_df, slots_df),
        "bounds": bounds,
        "minmax": "min",
        "verbose": True,
    }

    # Choose optimization algorithm based on the input
    if algorithm == "MPA":
        MPA_model = MPA.OriginalMPA(epoch=50, pop_size=50)
        best_agent = MPA_model.solve(problem)
    elif algorithm == "NRO":
        NRO_model = NRO.OriginalNRO(epoch=50, pop_size=50)
        best_agent = NRO_model.solve(problem)
        pass
    elif algorithm == "GA":
        GA_model = GA.BaseGA(epoch=50, pop_size=50)  # Use the correct GA class
        best_agent = GA_model.solve(problem)
        pass
    elif algorithm == "PSO":
        PSO_model = PSO.OriginalPSO(epoch=50, pop_size=50)
        best_agent = PSO_model.solve(problem)
        pass
    else:
        raise ValueError(f"Unknown algorithm: {algorithm}")

    return best_agent.solution, best_agent.target

# Warehouse sizes
warehouses = {
    "Small": {"num_items": 50, "num_slots": 35},
    "Medium": {"num_items": 100, "num_slots": 85},
    "Large": {"num_items": 150, "num_slots": 130},
}

# Number of runs to calculate average fitness
num_runs = 50
algorithm = "NRO"  # Set the desired optimization algorithm here (e.g., "MPA", "GA", "PSO")

# Run optimization for each warehouse size
results = {}
for size, params in warehouses.items():
    print(f"\n--- Optimizing for {size} Warehouse ---")
    items_df, slots_df = load_datasets(size)  # Load the datasets for the warehouse size

    fitness_values = []
    run_times = []

    for _ in range(num_runs):
        start_time = time.time()

        best_solution, best_fitness = optimize_allocation(
            items_df, slots_df, params["num_items"], params["num_slots"], algorithm=algorithm
        )

        run_time = time.time() - start_time
        run_times.append(run_time)

        # Append the raw fitness value
        fitness_values.append(best_fitness._fitness)
        print(best_fitness.__dict__)  # Print the attributes of the Target object

        # Create allocation result for each run
        allocation_result = pd.DataFrame({
            "ItemID": items_df["ItemID"],
            "AssignedSlot": [
                slots_df.iloc[int(slot_index)]["SlotID"] if slot_index < len(slots_df) else None
                for slot_index in best_solution
            ],
            "SlotType": [
                slots_df.iloc[int(slot_index)]["SlotType"] if slot_index < len(slots_df) else None
                for slot_index in best_solution
            ],
        })

    avg_fitness = np.mean(fitness_values)
    avg_runtime = np.mean(run_times)

    results[size] = {
        "avg_fitness": avg_fitness,
        "best_fitness": min(fitness_values),
        "allocation": allocation_result,
    }

    print(f"Best Fitness Score for {size} Warehouse: {min(fitness_values)}")
    print(f"Average Fitness Score for {size} Warehouse: {avg_fitness}")
    print(f"Average Runtime for {size} Warehouse: {avg_runtime} seconds")
    print(allocation_result.head())

# Save results to CSV files
from google.colab import files

for size, data in results.items():
    file_path = f"{size}_warehouse_allocation.csv"
    data["allocation"].to_csv(file_path, index=False)
    files.download(file_path)
    print(f"Downloaded {size} warehouse allocation as {file_path}")


INFO:mealpy.physics_based.NRO.OriginalNRO:Solving single objective optimization problem.



--- Optimizing for Small Warehouse ---
Datasets loaded for Small warehouse.


INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 1, Current best: 382.0, Global best: 382.0, Runtime: 0.64328 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 2, Current best: 363.0, Global best: 363.0, Runtime: 0.62515 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 3, Current best: 363.0, Global best: 363.0, Runtime: 0.62545 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 4, Current best: 363.0, Global best: 363.0, Runtime: 0.63247 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 5, Current best: 363.0, Global best: 363.0, Runtime: 0.64384 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 6, Current best: 363.0, Global best: 363.0, Runtime: 0.63517 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 7, Current best: 363.0, Global best: 363.0, Runtime: 0.62736 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 8, Cu

{'_objectives': array([350]), '_weights': array([1.]), '_fitness': 350.0}


INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 1, Current best: 367.0, Global best: 367.0, Runtime: 0.79944 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 2, Current best: 367.0, Global best: 367.0, Runtime: 1.00577 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 3, Current best: 367.0, Global best: 367.0, Runtime: 1.05293 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 4, Current best: 367.0, Global best: 367.0, Runtime: 1.00616 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 5, Current best: 367.0, Global best: 367.0, Runtime: 0.60909 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 6, Current best: 350.0, Global best: 350.0, Runtime: 0.62344 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 7, Current best: 350.0, Global best: 350.0, Runtime: 0.62611 seconds
INFO:mealpy.physics_based.NRO.OriginalNRO:>>>Problem: P, Epoch: 8, Cu

...

<IPython.core.display.Javascript object>

Downloaded Medium warehouse allocation as Medium_warehouse_allocation.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Downloaded Large warehouse allocation as Large_warehouse_allocation.csv
